In [1]:
import pandas as pd

df = pd.read_csv('dataset_mood_smartphone.csv')

In [2]:
del df['Unnamed: 0']

In [3]:
df.head()

,id,time,variable,value
0,AS14.01,2014-02-26 13:00:00.000,mood,6.0
1,AS14.01,2014-02-26 15:00:00.000,mood,6.0
2,AS14.01,2014-02-26 18:00:00.000,mood,6.0
3,AS14.01,2014-02-26 21:00:00.000,mood,7.0
4,AS14.01,2014-02-27 09:00:00.000,mood,6.0


In [4]:
test = df.loc[(df['variable'] == 'appCat.builtin')]
test.sort_values(by='value', ascending=False)

,id,time,variable,value
172955,AS14.14,2014-04-08 23:34:57.855,appCat.builtin,33960.246
204569,AS14.27,2014-04-20 23:13:11.269,appCat.builtin,31729.818
165330,AS14.12,2014-04-19 02:10:37.487,appCat.builtin,28163.993
163298,AS14.12,2014-04-09 02:42:21.210,appCat.builtin,27160.374
196457,AS14.26,2014-04-16 23:48:21.302,appCat.builtin,20556.690
...,...,...,...,...
229799,AS14.33,2014-04-26 04:32:01.187,appCat.builtin,0.001
182404,AS14.20,2014-04-12 22:53:05.420,appCat.builtin,0.001
162154,AS14.12,2014-04-02 16:42:16.523,appCat.builtin,-1.218
150040,AS14.02,2014-03-16 15:46:43.769,appCat.builtin,-44.689


In [5]:
display(df)

,id,time,variable,value
0,AS14.01,2014-02-26 13:00:00.000,mood,6.000
1,AS14.01,2014-02-26 15:00:00.000,mood,6.000
2,AS14.01,2014-02-26 18:00:00.000,mood,6.000
3,AS14.01,2014-02-26 21:00:00.000,mood,7.000
4,AS14.01,2014-02-27 09:00:00.000,mood,6.000
...,...,...,...,...
376907,AS14.30,2014-04-11 07:51:16.948,appCat.weather,8.032
376908,AS14.30,2014-04-19 11:00:32.747,appCat.weather,3.008
376909,AS14.30,2014-04-26 10:19:07.434,appCat.weather,7.026
376910,AS14.30,2014-04-27 00:44:48.450,appCat.weather,23.033


In [6]:
#Removing negative time values

df = df.drop(df[(df['variable'] != 'circumplex.arousal') & (df['variable'] != 'circumplex.valence') & (df['value'] < 0)].index)

# Dropping NaN values

#df = df.drop(df[df['value'].isna()].index)

# Replace NaN values 
means_valence = {1: -2.0,
 2: -2.0,
 3: -2.0,
 4: -1.0,
 5: -1.0,
 6: 0.0,
 7: 1.0,
 8: 1.0,
 9: 2.0,
 10: 2.0}

means_arousal = {1:1.0,
                 2: -2.0,3: -2.0,4: 1.0,5: 0.0, 6:0.0, 7:0.0, 8:1.0, 9:1.0, 10:2.0}

replacement_dict = {}
for i in range(len(df [ (df['variable']=='circumplex.arousal') & (df['value'].isna())])):
    id = df [ (df['variable']=='circumplex.arousal') & (df['value'].isna())].iloc[i]['id']
    time = df [ (df['variable']=='circumplex.arousal') & (df['value'].isna())].iloc[i]['time']

    index_of_row = df[(df['id']==id)&(df['time']==time)&(df['variable']=='circumplex.arousal')].index[0]

    mood = df [ (df['variable']=='mood') & (df['id'] == id) & (df['time'] == time)].iloc[0]['value']
    replacement_val = means_arousal[int(mood)]

    replacement_dict[index_of_row] = replacement_val

for index_of_row in replacement_dict.keys():
    df.at[index_of_row, 'value'] = replacement_dict[index_of_row]

replacement_dict = {}
for i in range(len(df [ (df['variable']=='circumplex.valence') & (df['value'].isna())])):
    id = df [ (df['variable']=='circumplex.valence') & (df['value'].isna())].iloc[i]['id']
    time = df [ (df['variable']=='circumplex.valence') & (df['value'].isna())].iloc[i]['time']

    index_of_row = df[(df['id']==id)&(df['time']==time)&(df['variable']=='circumplex.valence')].index[0]

    mood = df [ (df['variable']=='mood') & (df['id'] == id) & (df['time'] == time)].iloc[0]['value']
    replacement_val = means_arousal[int(mood)]

    replacement_dict[index_of_row] = replacement_val

for index_of_row in replacement_dict.keys():
    df.at[index_of_row, 'value'] = replacement_dict[index_of_row]

# Dropping lowly correlated and seemingly unimportant variables

df = df.drop(df[(df['variable'] == 'call')].index)
df = df.drop(df[(df['variable'] == 'appCat.travel')].index)
df = df.drop(df[(df['variable'] == 'appCat.weather')].index)
df = df.drop(df[(df['variable'] == 'appCat.utilities')].index)
df = df.drop(df[(df['variable'] == 'appCat.finance')].index)
df = df.drop(df[(df['variable'] == 'appCat.unknown')].index)


# Removed timed values under 3 seconds for opening and closing applications

df = df.drop(df[(df['variable'] != 'circumplex.arousal') & (df['variable'] != 'mood') & (df['variable'] != 'circumplex.sms') & (df['variable'] != 'circumplex.valence') & (df['variable'] != 'activity') & (df['value'] < 3)].index)

# Removes values with exact same time
df =  df.drop_duplicates(subset=['id', 'time', 'variable'], keep='first')




In [9]:
df[['date', 'time']] = df['time'].str.split(' ', n=1, expand=True)

In [10]:
df.to_csv('cleaned_nans_replaced.csv')